In [248]:
import numpy as np
import pandas as pd
from graphviz import Digraph

In [249]:
df = pd.read_csv("data/transfer_data/club_games.csv")
df = df.drop(columns=['game_id', 'club_id', 'own_manager_name', 'opponent_manager_name', 'opponent_id',],axis=1)
df

,own_goals,own_position,opponent_goals,opponent_position,hosting,is_win
0,0,NaN,2,NaN,Home,0
1,3,NaN,1,NaN,Home,1
2,4,NaN,5,NaN,Home,0
3,2,1.0,1,15.0,Home,1
4,0,18.0,2,11.0,Home,0
...,...,...,...,...,...,...
139265,4,NaN,0,NaN,Away,1
139266,0,8.0,1,16.0,Away,0
139267,1,7.0,2,6.0,Away,0
139268,2,NaN,0,NaN,Away,1


### Cleaning up data
1. Change datatype
2. Drop NaN rows
3. Remove irrelevant columns to decision tree (ie. manager name, club id)


#### 1. Change Datatypes
- Change data type of 'hosting' column from *object* to *boolean*
- Change data type of 'position' column from *float* to *int64*

In [250]:
df['hosting'] = df['hosting'].apply(lambda x : 1 if x == "Home" else 0).astype('Int64')
# df['is_win'] = df['is_win'].apply(lambda x : True if x == 1 else False).astype(bool)
df['opponent_position'] = df['opponent_position'].astype('Int64')
df['own_position'] = df['own_position'].astype('Int64')

Drop rows with any NaN values

In [251]:
df = df.dropna()
df.dtypes

own_goals            int64
own_position         Int64
opponent_goals       int64
opponent_position    Int64
hosting              Int64
is_win               int64
dtype: object

Now, remove irrelevant columns, like manager name, and clubid
We removed goals data as well, because uhm it is pretty obvious to compare with goals.

In [252]:
df

,own_goals,own_position,opponent_goals,opponent_position,hosting,is_win
3,2,1,1,15,1,1
4,0,18,2,11,1,0
5,2,13,0,3,1,1
6,1,15,1,1,1,0
7,0,7,2,6,1,0
...,...,...,...,...,...,...
139248,4,18,4,8,0,0
139249,1,16,1,3,0,0
139250,0,17,3,9,0,0
139266,0,8,1,16,0,0


## Node Class

In [253]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        """Constructor for a single node

		Args:
			feature_index (_type_, optional): _description_. Defaults to None.
			threshold (_type_, optional): _description_. Defaults to None.
			left (_type_, optional): _description_. Defaults to None.
			right (_type_, optional): _description_. Defaults to None.
			info_gain (_type_, optional): _description_. Defaults to None.
			value (_type_, optional): _description_. Defaults to None.
		"""
        # For Decision Nodes
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        
        # For Leaf Nodes
        self.value = value
        


In [254]:
from collections import Counter


class DecisionTreeClassifier():
    def __init__(self, min_samples_split=2, max_depth=2):
        # Init the root of the tree
        self.root = None
        
        # Stop Condition
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
    def build_tree(self, dataset, curr_depth=0):
        """Recursively build a decision tree

        Args:
            dataset (_type_): _description_
            curr_depth (int, optional): _description_. Defaults to 0.
        """
        
        # Split features and targets
        X, Y = dataset[:, :-1], dataset[:,:-1]
        
        # Extract num samples, num features
        num_samples , num_features = np.shape(X)
        
        # Split til stopping condition
        if num_samples >= self.min_samples_split and curr_depth <= self.max_depth:
            # Find best split
            best_split = self.get_best_split(dataset, num_samples, num_features)
            # Check if info gain is pos.
            # Because info gain < 0 means we are splitting a pure node (Node consisting of only one class...)
            # Alr ideal node
            if best_split['info_gain'] > 0:
                # recur left
                left_subtree = self.build_tree(best_split['dataset_left'], curr_depth + 1)
                # recur right
                right_subtree = self.build_tree(best_split['dataset_right'], curr_depth + 1)
                # Return decision node
                return Node(best_split["feature_index"], best_split["threshold"], left_subtree, right_subtree, best_split["info_gain"])
        
        # Stopping condition met
        # Compute Leaf Node
        leaf_value = self.calculate_leaf_value(Y)
        # Return leaf node
        return Node(value=leaf_value)
    
    def get_best_split(self, dataset, num_samples, num_features):
        """Function to find the best split

        Args:
            dataset (_type_): _description_
            num_samples (_type_): _description_
            num_features (_type_): _description_
        """
        
        # Dictionary to store the best split
        best_split = {}
        max_info_gain = -float("inf")
        
        # Iterate over all features
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            # Use unique to remove duplicate feature values and only iterate over unique feature values
            possible_thresholds = np.unique(feature_values)
            for threshold in possible_thresholds:
                # Iterate over all possible thresholds
                # Get current split
                dataset_left , dataset_right = self.split(dataset, feature_index, threshold)
                # Check if childs are not null (Check for empty child nodes)
                if len(dataset_left) > 0 and len(dataset_right) > 0:
                    # Extract only target values from datasets
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    # Compute information gain
                    curr_info_gain = self.information_gain(y, left_y, right_y, "gini")
                    # Update the best split if needed
                    if curr_info_gain > max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain
        
        # Return best split
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        """Function to split the data based on feature_index and threshold

        Args:
            dataset (_type_): _description_
            feature_index (_type_): _description_
            threshold (_type_): _description_
        """
        # Left Child
        dataset_left = np.array([row for row in dataset if row[feature_index] <= threshold])
        # Right Child
        dataset_right = np.array([row for row in dataset if row[feature_index] > threshold])
        
        return dataset_left, dataset_right
    
    def information_gain(self, parent, left_child, right_child, mode="entropy"):
        """Function to calculate information gain

        Args:
            parent (_type_): _description_
            left_child (_type_): _description_
            right_child (_type_): _description_
            mode (str, optional): _description_. Defaults to "entropy".
        """
        # Weight = Relative size of a child compared to parent
        left_weight = len(left_child) / len(parent)
        right_weight = len(right_child) / len(parent)
        
        if mode=="gini":
            gain = self.gini_index(parent) - (left_weight * self.gini_index(left_child) + right_weight * self.gini_index(right_child))
        else:
            gain = self.entropy(parent) - (left_weight * self.entropy(left_child) + right_weight * self.entropy(right_child))
            
        return gain
    
    def entropy(self, y):
        """Function to compute Entropy of our dataset Y

        Args:
            y (_type_): _description_
        """
        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len(y[y==cls]) / len(y)
            entropy += -1 * p_cls * np.log2(p_cls)
        
        return entropy

    def gini_index(self, y):
        """Function to calculate Gini Index

        Args:
            y (_type_): _description_
        """
        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            p_cls = len(y[y==cls]) / len(y)
            gini += p_cls ** 2
        
        return 1 - gini
        
    def calculate_leaf_value(self, Y):
        """Function to compute leaf node

        Args:
            Y (_type_): _description_
        """
        # Majority class in the node (Return most occuring class in Y)
        # Y = list(Y)
        # Flatten it
        Y = np.asarray(Y).flatten()
        Y = [int(y) for y in Y]
        # Handles when Y = boolean value list
        counts = Counter(Y)
        return counts.most_common(1)[0][0]
        # A=[0, 1]
        # if all(y in (0, 1) for y in Y):
        #     print("All ")
        #     counts = Counter(Y)
        #     if counts[0] >= counts[1]:
        #         return 0
        #     else:
        #         return 1
        # return max(set(Y), key=Y.count)
        # return max(Y, key=Y.count)
    
    def print_tree(self, tree=None, indent=" "):
        """Function to print out the decision tree

        Args:
            tree (_type_, optional): _description_. Defaults to None.
            indent (str, optional): _description_. Defaults to " ".
        """
        if not tree:
            tree = self.root
        
        if tree.value is not None:
            print(tree.value)
        else:
            print("X_" + str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain) 
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
            
    def fit(self, X, Y):
        """Funtion to train the tree

        Args:
            X (_type_): _description_
            Y (_type_): _description_
        """
        dataset = np.concatenate((X,Y), axis=1)
        self.root = self.build_tree(dataset)
        
    def predict(self, X):
        """Function to predict new dataset

        Args:
            X (_type_): _description_
        """
        predictions = [self.make_prediction(x, self.root) for x in X]
        return predictions
    
    def make_prediction(self, x, tree):
        """Function to predict a single data point

        Args:
            x (_type_): _description_
            tree (_type_): _description_
        """
        
        if tree.value != None:
            # If node == Leaf node, return value
            return tree.value
        # Not a leaf node
        # Extract feature value
        feature_value = x[tree.feature_index]
        if feature_value <= tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)
        
            
    

    def visualize_tree(self):
        def add_nodes_edges(tree, dot=None, node_id="0"):
            if dot is None:
                dot = Digraph()

            if tree.value is not None:
                dot.node(node_id, f"Leaf: {tree.value}")
            else:
                label = f"X_{tree.feature_index} <= {tree.threshold}\nGain: {tree.info_gain:.4f}"
                dot.node(node_id, label)

                left_id = node_id + "L"
                right_id = node_id + "R"
                dot.edge(node_id, left_id, "True")
                dot.edge(node_id, right_id, "False")

                add_nodes_edges(tree.left, dot, left_id)
                add_nodes_edges(tree.right, dot, right_id)

            return dot

        dot = add_nodes_edges(self.root)
        return dot


## Train Test-Split

In [255]:
X = df.iloc[:, :-1].values
Y = df.iloc[:, -1].values.reshape(-1, 1)
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test =train_test_split(X, Y, test_size=0.1, random_state=41)

### Fit the model

In [256]:
classifier = DecisionTreeClassifier(min_samples_split=30, max_depth=50)
classifier.fit(X_train, Y_train)
classifier.print_tree()
classifier.visualize_tree().render("tree", format="png", view=True)


X_0 <= 1 ? 0.175505140663064
 left:X_2 <= 0 ? 0.11265644996683777
  left:X_0 <= 0 ? 0.4963900375066127
    left:0
    right:1
  right:1
 right:X_2 <= 1 ? 0.24021028734615552
  left:1
  right:X_0 <= 2 ? 0.23856020855637367
    left:2
    right:X_2 <= 2 ? 0.29032693582608693
        left:2
        right:X_0 <= 3 ? 0.2435794148554739
                left:3
                right:X_2 <= 3 ? 0.26001525422824806
                                left:3
                                right:X_0 <= 4 ? 0.16666666666666657
                                                                left:4
                                                                right:5


'tree.png'

In [257]:
Y_pred = classifier.predict(X_test)
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, Y_pred)

0.40724444899212114